In [29]:
import pandas as pd
from pathlib import Path
import os
import importlib
import sys
PROJECT_ROOT = Path().resolve().parents[0]
sys.path.append(str(PROJECT_ROOT))


In [33]:

from src.data import load_data

In [34]:
importlib.reload(load_data)

<module 'src.data.load_data' from 'D:\\Code_ML\\energy_consumption_forecasting\\src\\data\\load_data.py'>

In [35]:
folder_path = "D:/Code_ML/energy_consumption_forecasting/src/data/raw"

utility_data = load_data.load_raw_data(Path(folder_path))


list(utility_data.keys())

['AEP', 'DAYTON', 'DOM', 'DUQ']

In [36]:
df_aep = utility_data["AEP"]

df_aep.columns

Index(['Datetime', 'MW'], dtype='object')

In [11]:
import pandas as pd


def add_time_features(df):
    df = df.copy()
    df["hour"] = df["Datetime"].dt.hour
    df["day_of_week"] = df["Datetime"].dt.dayofweek
    df["month"] = df["Datetime"].dt.month
    return df


def add_lag_features(df, lags=(1, 24, 168)):
    df = df.copy()
    for lag in lags:
        df[f"lag_{lag}"] = df["MW"].shift(lag)
    return df


def add_rolling_features(df, windows=(24, 168)):
    df = df.copy()
    for window in windows:
        df[f"rolling_mean_{window}"] = (
            df["MW"]
            .shift(1)
            .rolling(window)
            .mean()
        )
    return df


def build_features_for_utility(df):
    """
    Feature engineering for a single utility time series.
    """
    df = df.sort_values("Datetime")

    df = add_time_features(df)
    df = add_lag_features(df)
    df = add_rolling_features(df)

    # Drop NaNs created by lags and rolling windows
    df = df.dropna()

    return df

In [12]:
features_aep = build_features_for_utility(df_aep)

features_aep.head()

KeyError: 'MW'